In [2]:
import numpy as np
from pyqpanda_alg.QFinance import grover
from pyqpanda3.core import CPUQVM, QCircuit, QProg, TOFFOLI, Z, H, U1
from pyqpanda_alg.plugin import hadamard_circuit, QFT

In [3]:
# flip if x0 * x1 + x0 - x1 - current_min < 0
def flip_oracle_function(q_index_value, current_min):
    q_index = q_index_value[:2]
    q_value = q_index_value[2:]
    n_value = len(q_value)
    factor = np.pi * 2 ** (1 - n_value)
    cal_cir = QCircuit()
    cal_cir << hadamard_circuit(q_value)
    for i, q_i in enumerate(q_value):
        cal_cir << U1(q_i, factor * 2 ** i).control(q_index)
        cal_cir << U1(q_i, factor * 2 ** i).control(q_index[0])
        cal_cir << U1(q_i, -factor * 2 ** i).control(q_index[1])
        cal_cir << U1(q_i, factor * 2 ** i * (-current_min))
        cal_cir << QFT(q_value).dagger()
        return cal_cir
    
demo_search = grover.GroverAdaptiveSearch(init_value=0, n_index=2, oracle_circuit=flip_oracle_function)

In [4]:
def n_value_function(current_min):
    n_value = 2 if current_min == 0 else 3
    return n_value

def value_function(var_array):
    var_array = list(map(int, var_array))[::-1]
    value = var_array[0] * var_array[1] + var_array[0] - var_array[1]
    return value

In [5]:
res = demo_search.run(continue_times=3,
        n_value_function=n_value_function,
        value_function=value_function,
        process_show=True)
print(res)

======searching 1 ,rotation = 1 ======
======searching 2 ,rotation = 1 ======
Current minimum Key:  10
Current minimum Value:  -1
======searching 1 ,rotation = 1 ======
minimum Key Again:  10
minimum Value No Change:  -1
======searching 2 ,rotation = 1 ======
minimum Key Again:  10
minimum Value No Change:  -1
======searching 3 ,rotation = 1 ======
rotations:  5
([[0, 1]], -1)
